In [31]:
import os
import xml.etree.ElementTree as ET

def split_xml_file(xml_file, max_size=2*1024*1024):  # 2 МБ в байтах
    """
    Разделяет XML-файл на заголовок и составные файлы размером не более max_size байт.

    Args:
        xml_file: Путь к XML-файлу.
        max_size: Максимальный размер составных файлов в байтах (по умолчанию 2 МБ).

    Returns:
        Кортеж из путей к заголовочному и составным файлам.
    """

    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Создаём заголовочный файл
    header_file = xml_file.replace(".xml", "_header.xml")
    header_content = []
    for element in root.iter():
        header_content.append(f"<{element.tag}>")
        header_content.append(f"</{element.tag}>")
    print(header_content)
    header_tree = ET.ElementTree(ET.fromstring(f"<root>{''.join(header_content[1:])}</root>"))
    header_tree.write(header_file, encoding='utf-8', xml_declaration=True)

    # Создаём составные файлы
    parts_files = []
    current_size = 0
    current_part = 1
    current_part_file = xml_file.replace(".xml", f"_part_{current_part}.xml")
    parts_files.append(current_part_file)
    current_part_tree = ET.ElementTree(ET.Element('root'))  # Создаём дерево для составных файлов

    for element in root.iter():
        element_text = element.text if element.text is not None else ''
        element_size = len(ET.tostring(element, encoding='utf-8'))
        if current_size + element_size > max_size:
            # Сохраняем текущий составной файл
            current_part_tree.write(current_part_file, encoding='utf-8', xml_declaration=True)

            # Переходим к следующему составному файлу
            current_part += 1
            current_part_file = xml_file.replace(".xml", f"_part_{current_part}.xml")
            parts_files.append(current_part_file)
            current_part_tree = ET.ElementTree(ET.Element('root'))
            current_size = element_size

        current_size += element_size
        current_part_tree.getroot().append(ET.fromstring(f"<{element.tag}>{element_text}</{element.tag}>"))

    # Сохраняем последний составной файл
    current_part_tree.write(current_part_file, encoding='utf-8', xml_declaration=True)

    return header_file, parts_files

# Пример использования
xml_file = "example.xml"  # Замените на путь к вашему XML-файлу
header_file, parts_files = split_xml_file(xml_file)

print(f"Заголовочный файл: {header_file}")
print(f"Составные файлы: {parts_files}")

['<root>', '</root>', '<header>', '</header>', '<title>', '</title>', '<data>', '</data>', '<item>', '</item>', '<item>', '</item>', '<item>', '</item>', '<item>', '</item>', '<item>', '</item>', '<item>', '</item>', '<item>', '</item>', '<item>', '</item>', '<item>', '</item>', '<item>', '</item>', '<item>', '</item>', '<item>', '</item>', '<item>', '</item>', '<item>', '</item>', '<item>', '</item>', '<item>', '</item>', '<item>', '</item>', '<item>', '</item>', '<item>', '</item>', '<item>', '</item>', '<item>', '</item>', '<item>', '</item>']


ParseError: junk after document element: line 1, column 13 (<string>)

In [76]:
import xml.etree.ElementTree as ET
import os

def split_xml(input_file, output_dir = 'split_files', max_size=2*1024*1024):
    """
    Разделяет XML-файл на заголовочный и составные файлы размером 2 Мб.

    Args:
      input_file: Путь к входному XML-файлу.
      output_dir: Каталог для сохранения выходных файлов (по умолчанию "split_files").
    """
    os.makedirs(output_dir, exist_ok=True)
  # Чтение XML-файла
    tree = ET.parse(input_file)
    root = tree.getroot()

  # Создание заголовочного файла
    header_file = os.path.join(output_dir, "header.xml")
    with open(header_file, "w") as f:
        f.write(f"  <root>\n")
    # Вызов функции write_tags для записи тегов
        write_tags(root, f)  # Передаем корневой элемент и файл
        f.write(f"  </root>\n")

  # Создание составных файлов
    current_part = 1
    current_part_file = os.path.join(output_dir, f"content_{current_part}.xml")
    for element in root.iter():
        element_text = element.text if element.text is not None else ''
        current_part_tree = ET.ElementTree(ET.fromstring(f"<{element.tag}>{element_text}</{element.tag}>"))
        current_part_tree.write(current_part_file, encoding='utf-8', xml_declaration=True)
        # Переходим к следующему составному файлу
        current_part += 1
        current_part_file = os.path.join(output_dir, f"content_{current_part}.xml")
        

    print(f"XML-файл успешно разделен на {current_part} составных файлов в каталоге {output_dir}.")

# Рекурсивная функция для записи тегов
def write_tags(element, file):
  """
  Рекурсивно записывает теги в файл.

  Args:
    element: Текущий элемент XML.
    file: Файл для записи.
  """
  for child in element:
    file.write(f"  <{child.tag}>\n")
    print(child.tag)
    write_tags(child, file)
    file.write(f"  </{child.tag}>\n")

split_xml('example.xml')

header
title
data
item
item
item
item
item
item
item
item
item
item
item
item
item
item
item
item
item
item
item
item
item
item
XML-файл успешно разделен на 27 составных файлов в каталоге split_files.
